## Part c)

Implement now the variational quantum eigensolver (VQE) for the above
Hamiltonian and set up the circuit(s) which is(are) needed in order to find
the eigenvalues of this system. Discuss the results and compare these
with those from part b). Feel free to use either **Qiskit** or your own
code (based on the setup from part a)) or both approaches. Discuss
your results.

## The VQE algoritm

The VQE algorithm consists of several steps, partially done on a classical computer:
1. A parameterized ansatz for the quantum state is implemented on a quantum computer.

2. The ansatz is measured in a given measurement basis.

3. Postprocessing on a classical computer converts the measurement outcomes to an expectation value.

4. Classical minimization algorithms are used to up- date the variational parameters.

In [12]:
import qiskit as qk
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
import numpy as np
from qiskit import transpile

# initialize constants for quantum circuit
nqubits = 1
ncbits = 1
qreg = qk.QuantumRegister(nqubits)
creg = qk.ClassicalRegister(ncbits)
# Initialize a simulator
simulator = AerSimulator()

# make circuit for measurement of the Z term - default basis is z basis so no changes needed
def measureZ(state):
    z = qk.QuantumCircuit(qreg, creg)
    z.initialize(state, qreg)
    z.z(0)
    z.measure(qreg[0], creg[0])
    new_circuit = transpile(z, simulator)
    job = simulator.run(new_circuit, shots = 1000)
    results = job.result().get_counts(z)
    return results
    

# make circuit for measurement of the X term - need to transfer basis with hadamard gate
def measureX(state):
    x = qk.QuantumCircuit(qreg,creg)
    x.initialize(state, qreg)
    x.h(0)
    x.z(0)
    x.h(0)
    x.measure(qreg[0],creg[0])
    new_circuit = transpile(x, simulator)
    job = simulator.run(new_circuit, shots = 1000)
    results = job.result().get_counts(x)
    return results

# make circuit for measurement of the I term
def measureI(state):
    i = qk.QuantumCircuit(qreg,creg)
    i.initialize(state, qreg)
    i.measure(qreg[0],creg[0])
    new_circuit = transpile(i, simulator)
    job = simulator.run(new_circuit, shots = 1000)
    results = job.result().get_counts(i)
    return results
    
measureZ(np.array([1,0]))

{'0': 1000}

In [19]:
# Pauli Matrices
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
I = np.array([[1,0],[0,1]])

# Express Hamiltonian with Pauli Matrices
dim = 2
Hamiltonian = np.zeros((dim,dim))
e0 = 0.0; e1 = 4.0; V11 = 3; V22 = -3; V12 = 0.2; V21 = 0.2
c = (V11+V22)/2; omegaz = (V11-V22)/2; omegax = V12
epsilon = (e0+e1)/2; Omega = (e0-e1)/2

Hamiltonian = (epsilon+c)*I+(omegaz+Omega)*Z+omegax*X

# Calculate Eigenvalues with numpy to have a reference value
EigValues = np.zeros(dim)
EigValues, EigVectors = np.linalg.eig(Hamiltonian)
print("Eigenvalues with numpy: ",EigValues)


# Rotation Operators
def Rx(theta):
    return np.cos(theta*0.5)*I-1j*np.sin(theta*0.5)*X
def Ry(phi):
    return np.cos(phi*0.5)*I-1j*np.sin(phi*0.5)*Y

# Basis states
b0 = np.array([1,0])
b1 = np.array([0,1])

# expectation value of energy
def energy(theta,phi):
    Basis = Ry(phi) @ Rx(theta)@b0
    # perform measurments
    resultZ = measureZ(Basis)
    resultX = measureX(Basis)
    resultI = measureI(Basis)
    print(resultZ)

    # Convert measurement results to probabilities
    total_counts = sum(resultZ.values())
    print(total_counts)
    probZ = resultZ.get('0', 0) / total_counts
    probX = resultX.get('0', 0) / total_counts
    probI = resultI.get('0', 0) / total_counts

    #calculate energies
    energyZ = (omegaz+Omega)*probZ
    energyX = (omegax)* probX
    energyI = (epsilon+c)*probI
    return energyZ + energyX + energyI

# make gradient descend measurments
eta = 0.1
Niterations = 100
# Random angles using uniform distribution
theta = 2*np.pi*np.random.rand()
phi = 2*np.pi*np.random.rand()
pi2 = 0.5*np.pi
for iter in range(Niterations):
    thetagradient = 0.5*(energy(theta+pi2,phi)-energy(theta-pi2,phi))
    phigradient = 0.5*(energy(theta,phi+pi2)-energy(theta,phi-pi2))
    theta -= eta*thetagradient
    phi -= eta*phigradient
    #print(energy(theta,phi))
print(energy(theta,phi))


Eigenvalues with numpy:  [3.0198039 0.9801961]
{'1': 625, '0': 375}
{'0': 630, '1': 370}
{'1': 412, '0': 588}
{'0': 444, '1': 556}
{'0': 336, '1': 664}
{'0': 644, '1': 356}
{'1': 404, '0': 596}
{'1': 603, '0': 397}
{'1': 671, '0': 329}
{'1': 329, '0': 671}
{'1': 392, '0': 608}
{'1': 641, '0': 359}
{'1': 708, '0': 292}
{'0': 710, '1': 290}
{'1': 363, '0': 637}
{'1': 610, '0': 390}
{'1': 722, '0': 278}
{'0': 722, '1': 278}
{'0': 589, '1': 411}
{'1': 625, '0': 375}
{'1': 755, '0': 245}
{'0': 750, '1': 250}
{'0': 606, '1': 394}
{'1': 627, '0': 373}
{'1': 770, '0': 230}
{'0': 770, '1': 230}
{'1': 334, '0': 666}
{'1': 668, '0': 332}
{'1': 824, '0': 176}
{'0': 797, '1': 203}
{'1': 364, '0': 636}
{'1': 669, '0': 331}
{'1': 810, '0': 190}
{'0': 827, '1': 173}
{'0': 640, '1': 360}
{'1': 620, '0': 380}
{'1': 871, '0': 129}
{'0': 850, '1': 150}
{'0': 639, '1': 361}
{'0': 348, '1': 652}
{'1': 867, '0': 133}
{'0': 878, '1': 122}
{'1': 385, '0': 615}
{'1': 636, '0': 364}
{'1': 867, '0': 133}
{'0': 87